In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

plt.style.use('seaborn-colorblind')

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

from acquire_walkthrough import get_iris_data
from prepare_walkthrough import prep_iris

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'acquire_walkthrough'

In [2]:
df = get_iris_data()
encoder, train, test = prep_iris(df)
train.head()

NameError: name 'get_iris_data' is not defined